公式Example

深層学習フレームワークには公式に様々なモデルのExampleコードが公開されています。


【問題1】公式チュートリアルモデルを分担して実行
TensorFLowの公式チュートリアルモデルを分担して実行してください。


以下の中から1人ひとつ選び実行し、その結果を簡単に発表してください。


models/tutorials at master · tensorflow/models



【問題2】（アドバンス課題）様々な手法を実行
TensorFLowやGoogle AI ResearchのGitHubリポジトリには、定番のモデルから最新のモデルまで多様なコードが公開されています。これらから興味あるものを選び実行してください。


なお、これらのコードは初学者向けではないため、巨大なデータセットのダウンロードが必要な場合など、実行が簡単ではないこともあります。そういった場合は、コードリーディングを行ってください。


models/research at master · tensorflow/models


google-research/google-research: Google AI Research


更新日が古いものはPythonやTensorFlowのバージョンが古く、扱いずらい場合があります。新しいものから見ることを推奨します。

異なるフレームワークへの書き換え

「ディープラーニングフレームワーク1」で作成した4種類のデータセットを扱うTensorFLowのコードを異なるフレームワークに変更していきます。


Iris（Iris-versicolorとIris-virginicaのみの2値分類）
Iris（3種類全ての目的変数を使用して多値分類）
House Prices
MNIST

Kerasへの書き換え
KerasはTensorFLowに含まれるtf.kerasモジュールを使用してください。


KerasにはSequentialモデルかFunctional APIかなど書き方に種類がありますが、これは指定しません。



【問題3】Iris（2値分類）をKerasで学習
TensorFlowによるIrisデータセットに対する2値分類をKerasに書き換えてください。

In [7]:

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import tensorflow as tf

In [8]:
from sklearn.datasets import load_iris
iris = load_iris()#irisデータ
#二次元の表形式のデータ（テーブルデータ）、irisのdataと特徴量をdf化
df = pd.DataFrame(data = iris.data , columns = iris.feature_names)
#dfにtarget列を作成しtargetのデータを格納
df['target'] = iris.target
# df.loc[df['target'] == 0]
# df.loc[df['target'] == 1]
# df.loc[df['target'] == 2]

In [9]:
#dfの50行から最後まで、特徴量までをXとする
X = df.iloc[50: , :4].values

#dfの50行から最後まで、targetをyとする
y = df.iloc[50: , -1].values

#yの2を０にして、0,1にして2値分類の形に整える
y[y==2]=0

#train_test_splitでXとyをトレインデータ：テストデータ＝8:2で分ける
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(
               X , y , test_size = 0.2)

#train_test_splitでX_trainとy_trainをトレインデータ：バリデーションデータ=8:2で分ける
X_train, X_val, y_train, y_val = train_test_split(
               X_train, y_train, test_size=0.2, random_state=0)

In [10]:
#keras
#Kerasパッケージは単体では動かす事ができない。バックエンドとしてTensorFlowなどの低レベルニューラルネットワークライブラリが必要
#from keras.modelsというのはモデルが色々はいったモジュール
from keras.models import Sequential
#レイヤーはkeras.layersにまとめられていて、その中のDenseを使う事を上で宣言
from keras.layers import Dense , Activation
from keras.optimizers import Adam
from keras.layers import Conv2D , MaxPooling2D , Flatten

In [11]:
from keras import backend as K
K.clear_session( )

#何も設定されていない真っ新なSequentialモデルを作るには、単純にSequential()
model = Sequential()

#Sequentialが持っているaddメソッドでレイヤーを追加、addメソッドの引数にDenseレイヤーを与える
#unitsはDenseを作る時に唯一必須のパラメータ
#入力値とは解析したいデータそのものの事、入力シェイプにはこの入力値の次元数を与える
#２つ目以降のDenseにinput_shapeは不要。
#Denseは全結合ニューラルネットワークなので、自動的に次の全てのunitsに接続される。
#model.add(Dense(units=1 , input_shape = (4 , )))

#2クラス分類、sigmoid
#model.add(Activation("sigmoid")) #model.add(Dense(units=1 , input_shape = (4 , ), activation="sigmoid"))
model.add(Dense(units=1 , input_shape = (4 , ),activation="sigmoid")) 

#モデルの要約を出力します．utils.print_summaryへのショートカット
#作成したニューラルネットワークのモデル形状の概要を表示
#Model: "sequential_1"の部分の表示
model.summary()

#訓練プロセスを作る（コンパイル：compile）
#損失関数及び正解に近づける修正方法である最適化を選択するのが「訓練プロセスの作成」でSequentialが持つcompile関数で設定
#loss=損失関数で、名前を文字列で指定（もしくは誤差オブジェクトを直接指定)
#optimizerには最適化方法を名前で指定(もしくは最適化オブジェクトを直接指定）
#metricsは評価関数：accuracy
#2クラス分類なので二値交差エントロピー
model.compile(loss = "binary_crossentropy" , 
             optimizer = Adam(lr = 0.01) , 
             metrics = ["accuracy"])

#訓練プロセス（コンパイル）を完了し、いよいよモデルの学習を開始する。
#学習するためには「入力シェイプで指定した次元数がある入力データ」と「その入力データで期待する答え」が1対となったデータの配列が必要
#Kerasの慣習的に入力データはx_train、期待する答えはy_trainと表す
#epochsはx_trainの入力データ全部を1塊として、その塊を学習し直す回数を指定。
#同じデータセットを何度も再学習させる事でモデル内のパラメータ（＝重み）をそのデータセットに合うよう自動的に調整される。
#batch_sizeはx_trainを小分けにする場合に与え、その小分けにした1セットを「サブバッチ」と呼び、「過学習」を防ぐ。
#fitメソッドの戻り値はHistoryオブジェクトとなり、フィッティングの結果を保持
#fitメソッドを実行すると、pythonのコンソール画面にその学習結果が1epoch毎に出力
#重要なのが「loss」、これはコンパイル時に引数で与えたloss（損失関数）が出した正解とのズレの値
#この値が小さい程訓練データに対して良くフィッティングしている事を意味する。

history = model.fit(X_train , y_train , 
                   batch_size = 1 , 
                   epochs = 100 , 
                   verbose = 1)
#lossが減っているので訓練データに良くフィットしていると思う。

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1)                 5         
Total params: 5
Trainable params: 5
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/100
64/64 [==============================] - 1s 9ms/step - loss: 2.3814 - accuracy: 0.5000
Epoch 2/100
64/64 [==============================] - 0s 5ms/step - loss: 0.6916 - accuracy: 0.5312
Epoch 3/100
64/64 [==============================] - 0s 3ms/step - loss: 0.6806 - accuracy: 0.5469
Epoch 4/100
64/64 [==============================] - 0s 3ms/step - loss: 0.6953 - accuracy: 0.5156
Epoch 5/100
64/64 [==============================] - 0s 3ms/step - loss: 0.6374 - accuracy: 0.5469
Epoch 6/100
64/64 [==============================] - 0s 3ms/step

64/64 [==============================] - 0s 2ms/step - loss: 0.2306 - accuracy: 0.9375
Epoch 73/100
64/64 [==============================] - 0s 2ms/step - loss: 0.2371 - accuracy: 0.9531
Epoch 74/100
64/64 [==============================] - 0s 2ms/step - loss: 0.2338 - accuracy: 0.9531
Epoch 75/100
64/64 [==============================] - 0s 2ms/step - loss: 0.2356 - accuracy: 0.9688
Epoch 76/100
64/64 [==============================] - 0s 2ms/step - loss: 0.2323 - accuracy: 0.9531
Epoch 77/100
64/64 [==============================] - 0s 2ms/step - loss: 0.2289 - accuracy: 0.9531
Epoch 78/100
64/64 [==============================] - 0s 2ms/step - loss: 0.2240 - accuracy: 0.9844
Epoch 79/100
64/64 [==============================] - 0s 2ms/step - loss: 0.2215 - accuracy: 0.9844
Epoch 80/100
64/64 [==============================] - 0s 2ms/step - loss: 0.2207 - accuracy: 0.9688
Epoch 81/100
64/64 [==============================] - 0s 2ms/step - loss: 0.2229 - accuracy: 0.9375
Epoch 82/100


In [12]:
from sklearn.metrics import precision_score

#Sequencialが持つpredictメソッド
y_pred_expect= model.predict(X_val)[: , 0]#全ての行の0列目を取得=0列目を行に変換

y_pred = np.where(y_pred_expect > 0.5 , 1 , 0)#0.5を超えている値を1、下回ると0mを取得
print("y_pred_expect" , y_pred_expect)
print("y_pred" , y_pred)
print("精度" , precision_score(y_val , y_pred))#yのバリデーションとy-predの精度

y_pred_expect [0.03793284 0.20415786 0.52261084 0.15046191 0.71961474 0.9406971
 0.13020998 0.1105088  0.9301784  0.7962742  0.32810578 0.25937724
 0.8958205  0.9401467  0.0311316  0.87506694]
y_pred [0 0 1 0 1 1 0 0 1 1 0 0 1 1 0 1]
精度 0.875


In [13]:
model.predict(X_val)
#valは16
#testは20

array([[0.03793284],
       [0.20415786],
       [0.52261084],
       [0.15046191],
       [0.71961474],
       [0.9406971 ],
       [0.13020998],
       [0.1105088 ],
       [0.9301784 ],
       [0.7962742 ],
       [0.32810578],
       [0.25937724],
       [0.8958205 ],
       [0.9401467 ],
       [0.0311316 ],
       [0.87506694]], dtype=float32)

【問題4】Iris（多値分類）をKerasで学習
TensorFlowによるIrisデータセットに対する3値分類をKerasに書き換えてください。

In [14]:
df = pd.DataFrame(data = iris.data , columns = iris.feature_names)
df['target'] = iris.target

X = df.iloc[: , :4].values
y = df.iloc[: , -1].values
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y = enc.fit_transform(y[:, np.newaxis])#np.newaxisで新たな次元の追加

X_train , X_test , y_train , y_test = train_test_split(
               X , y , test_size = 0.2)
X_train , X_val , y_train , y_val = train_test_split(
                X_train , y_train , test_size = 0.2)

In [15]:
K.clear_session( )

#ネットワーク構造をkeras.layersで定義する部分の２つを書いておいて、入力と出力がいくつあるのかkeras.Model()で定義
input_data = tf.keras.layers.Input(shape=(4 , ))

#学習データとテストデータのようにkerasの外からkerasモデルに渡すデータは必ず最初にkeras.layers.Input()で受け取る
x = tf.keras.layers.Dense(10 , activation=tf.nn.relu)(input_data)

#加える層の右にその層への入力を（）付きで与えるように書いて、1層ずつ増やす
#多クラス分類：reluを活性化関数
x = tf.keras.layers.Dense(10 , activation=tf.nn.relu)(x)
x = tf.keras.layers.Dense(10 , activation=tf.nn.relu)(x)

#多クラス分類なのでソフトマックス
output = tf.keras.layers.Dense(3 , activation=tf.nn.softmax)(x)
#入力と出力がいくつあるのかkeras.Model()で定義、今回は一個ずつ
model = tf.keras.Model(inputs = input_data , outputs = output)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [16]:
model.summary()
#categorical_crossentropyはクロスエントロピー、多クラス分類
model.compile(loss="categorical_crossentropy" , 
             optimizer = tf.train.AdamOptimizer(learning_rate=0.01) , 
             metrics=["accuracy"])

history = model.fit(X_train , y_train , 
                   batch_size = 1 , 
                   epochs = 50 , 
                   verbose=1)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense (Dense)                (None, 10)                50        
_________________________________________________________________
dense_1 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 33        
Total params: 303
Trainable params: 303
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
96/96 [==============================] - 0s 3ms/sample - loss: 0.5188 - acc: 0.8333
Epoch 2/50
96/96 [================

In [17]:
from sklearn.metrics import classification_report
y_pred_expect_2 = model.predict(X_val)
y_pred = np.where(y_pred_expect_2 > 0.5 , 1 , 0)
print(classification_report(y_val , y_pred, target_names=iris.target_names))

#クラス分類をそういった情報を簡単に要約して出力

             precision    recall  f1-score   support

     setosa       1.00      1.00      1.00         7
 versicolor       1.00      1.00      1.00         9
  virginica       1.00      1.00      1.00         8

avg / total       1.00      1.00      1.00        24



【問題5】House PricesをKerasで学習
TensorFlowによるHouse Pricesデータセットに対する回帰をKerasに書き換えてください。



In [18]:
df = pd.read_csv("train.csv")
X = df.loc[: , ["GrLivArea" , "YearBuilt"]]
y = df.loc[: , "SalePrice"]
y = np.log(y)
X_train , X_test , y_train , y_test = train_test_split(
                X , y , test_size=0.2)
X_train , X_val , y_train , y_val = train_test_split(
                X_train , y_train , test_size = 0.2)

In [19]:
K.clear_session( )

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10 , activation = tf.nn.relu , input_shape = (2 , )))
model.add(tf.keras.layers.Dense(10 , activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(10 , activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(1 , ))

#回帰、平均二乗誤差
model.compile(loss="mean_squared_error" , 
             optimizer = tf.train.AdamOptimizer(learning_rate=0.01))

history = model.fit(X_train , y_train , 
                   batch_size = 1 , 
                   epochs = 10 , 
                   verbose = 1 , 
                   validation_data = (X_train , y_train))

Train on 934 samples, validate on 934 samples
Epoch 1/10
934/934 [==============================] - 4s 4ms/sample - loss: 3183.5888 - val_loss: 8.2273
Epoch 2/10
934/934 [==============================] - 4s 4ms/sample - loss: 3.9348 - val_loss: 1.8929
Epoch 3/10
934/934 [==============================] - 4s 4ms/sample - loss: 2.0198 - val_loss: 0.6007
Epoch 4/10
934/934 [==============================] - 4s 4ms/sample - loss: 7.8112 - val_loss: 4.4395
Epoch 5/10
934/934 [==============================] - 4s 5ms/sample - loss: 17.5261 - val_loss: 0.6717
Epoch 6/10
934/934 [==============================] - 3s 3ms/sample - loss: 0.9846 - val_loss: 0.4830
Epoch 7/10
934/934 [==============================] - 4s 5ms/sample - loss: 0.6416 - val_loss: 0.2324
Epoch 8/10
934/934 [==============================] - 4s 4ms/sample - loss: 0.5494 - val_loss: 0.4379
Epoch 9/10
934/934 [==============================] - 9s 9ms/sample - loss: 60.1127 - val_loss: 0.1630
Epoch 10/10
934/934 [==========

【問題6】MNISTをKerasで学習
TensorFlowによるMNISTデータセットによる画像の多値分類をKerasに書き換えてください。

In [20]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(-1 , 784)
X_test = X_test.reshape(-1 , 784)
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255
# y_train = tf.one_hot(y_train , depth=3 ,dtype=None)
# y_test = tf.one_hot(y_test , depth=3 , dtype=None)
X_train = X_train.reshape(-1 , 28 , 28 , 1)
X_test = X_test.reshape(-1 , 28 , 28 , 1)
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train = enc.fit_transform(y_train[:, np.newaxis])
y_test = enc.transform(y_test[:, np.newaxis])

from sklearn.model_selection import train_test_split
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [21]:
K.clear_session( )

input_data = tf.keras.layers.Input(shape=(28 , 28 , 1))
con1 = tf.keras.layers.Conv2D(3 , kernel_size = (2,2)  , activation = tf.nn.relu)(input_data)
max_p1 = tf.keras.layers.MaxPooling2D((3,3) , strides=(1,1))(con1)
con2 = tf.keras.layers.Conv2D(3 , (2,2) , activation = tf.nn.relu)(max_p1)
max_p2 = tf.keras.layers.MaxPooling2D((3,3) , strides = (2,2))(con2)
fla = tf.keras.layers.Flatten()(max_p2)
x = tf.keras.layers.Dense(100 , activation=tf.nn.relu)(fla)
x = tf.keras.layers.Dense(50 , activation=tf.nn.relu)(x)
x = tf.keras.layers.Dense(20 , activation=tf.nn.relu)(x)
output = tf.keras.layers.Dense(10 , activation=tf.nn.softmax)(x)
model = tf.keras.Model(inputs = input_data , outputs = output)

In [22]:
model.summary()
model.compile(loss="categorical_crossentropy" , 
             optimizer = tf.train.AdamOptimizer(learning_rate=0.01) , 
             metrics=["accuracy"])

history = model.fit(X_train , y_train , 
                   batch_size = 100 , 
                   epochs = 10 , 
                   verbose=1)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 27, 27, 3)         15        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 25, 25, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 3)         39        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 11, 11, 3)         0         
_________________________________________________________________
flatten (Flatten)            (None, 363)               0         
_________________________________________________________________
dense (Dense)                (None, 100)               36400 